<a href="https://colab.research.google.com/github/sipocz/pid_time_series/blob/main/PID_NN_LSTM_230421/PID_NN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb==0.10.11

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import os
import seaborn as sns
import wandb
import numpy as np

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#user = "Anna"
user = "SL"

In [6]:
if user=="Anna":
    path = "/content/drive/MyDrive/Egyetem_MSc/Diplomamunka/2022Anna/Datapipeline/SINT3_415/3D_transposed/"
else:
    path = "/content/drive/MyDrive/2022Anna/Datapipeline/SINT3_415/3D_transposed/"

In [7]:
#user = "Anna"
user = "SL"
uzem = "Szint3"
data_source="SINT3_415/3D_transposed"
#fname="72C03_TC_error_toNN.csv"
fname_good = "415_SC_3D_part"
fname_bad = "415_SC_3D_part"
fname_good_ext=["1.csv","3.csv","5.csv","7.csv"]
fname_bad_ext=["2.csv","4.csv","6.csv","8.csv"]

In [8]:
# Elérési út a 415_SC_error-hoz
if user=="Anna":
    path_good = "/content/drive/MyDrive/Egyetem_MSc/Diplomamunka/2022Anna/Datapipeline/" + data_source + "/" + fname_good
    path_bad = "/content/drive/MyDrive/Egyetem_MSc/Diplomamunka/2022Anna/Datapipeline/" + data_source + "/" + fname_bad
    path_fig = "/content/drive/MyDrive/Egyetem_MSc/Diplomamunka/2022Anna/Datapipeline/plots/"
else:
    path_good = "/content/drive/MyDrive/2022Anna/Datapipeline/" + data_source + "/" + fname_good 
    path_bad = "/content/drive/MyDrive/2022Anna/Datapipeline/" + data_source + "/" + fname_bad
    path_fig = "/content/drive/MyDrive/2022Anna/Datapipeline/plots/"


In [9]:
print(path_good)
print(path_bad)

/content/drive/MyDrive/2022Anna/Datapipeline/SINT3_415/3D_transposed/415_SC_3D_part
/content/drive/MyDrive/2022Anna/Datapipeline/SINT3_415/3D_transposed/415_SC_3D_part


In [10]:
cols=[str(i) for i in range(60)]

In [11]:
#Listát készít a driveon lévő mappa exceleinek elérési útvonaláról (debug módban kiirja az elérési utakat)
def create_file_list(path_name, extension=".csv",debug=False):
 
  dir_path = os.path.dirname(path_name)
  eleresi_utak = []

  for root, dirs, files in os.walk(dir_path):
    for file in files:
      if file.endswith(extension):
        if debug:
          print(root+'/'+str(file))
        eleresi_utak.append(root+'/'+str(file))
  return (eleresi_utak)

In [12]:
#File lista meghívás
file_list=create_file_list(path, ".csv")

In [13]:
df_good=pd.DataFrame(columns=cols)
df_bad=pd.DataFrame(columns=cols)
print(df_good.head())
for i,goods in enumerate(fname_good_ext):
    df_good_tmp = pd.read_csv(path_good+goods,usecols=None )
    df_good_tmp.columns=cols
    df_good=pd.concat([df_good,df_good_tmp],axis=0,)
    df_bad_tmp = pd.read_csv(path_bad+fname_bad_ext[i],usecols=None)
    df_bad_tmp.columns=cols
    df_bad=pd.concat([df_bad,df_bad_tmp],axis=0)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
Index: []

[0 rows x 60 columns]


In [14]:
len(df_good)

2646

In [15]:
print(df_good.isnull().values.any())
print(df_bad.isnull().values.any())

False
False


In [16]:

import matplotlib.pyplot as plot

df_ = df_good

# You must normalize the data before applying the fit method
df_good_normalized=(df_ - df_.mean()) / df_.std()

# Normalize bad data with the good data parameters
df_bad_normalized=(df_bad - df_.mean()) / df_.std()

In [17]:
def series_check(act):
  count = 0
  for i in range(len(act)):
    if act[i] < 0.2 and act[i] > -0.2:
      count +=1
  if count == 20: return True
  else: return False

In [18]:
df_good_normalized["state"]=0
df_bad_normalized["state"]=1

In [19]:
for i in range(len(df_bad_normalized)):
    act=df_bad_normalized.iloc[i,40:60]

result=series_check(act)
if result==True:
    df_bad_normalized.iloc[i,-1]=0
else:
    df_bad_normalized.iloc[i,-1]=1


In [20]:
df_all_normalized=pd.concat([df_good_normalized,df_bad_normalized],axis=0)

In [21]:
# Ez jó, kiinduló csv sora 60 széles, egymás után Sp Out Error
X_good = np.ndarray(shape=(len(df_good),20,3))
X_good[:,:,0] = df_good_normalized.iloc[:, 0:20]
X_good[:,:,1] = df_good_normalized.iloc[:, 20:40]
X_good[:,:,2] = df_good_normalized.iloc[:, 40:60]


In [22]:
y_all=df_all_normalized["state"]


In [23]:
X_all = np.ndarray(shape=(len(df_all_normalized),20,3))
X_all[:,:,0] = df_all_normalized.iloc[:, 0:20]
X_all[:,:,1] = df_all_normalized.iloc[:, 20:40]
X_all[:,:,2] = df_all_normalized.iloc[:, 40:60]

In [24]:
_N1_=1100 #70  #700
_N2_=118 #12  #120
_lr_=0.001
_batch_size_=32
_drop1_=0.5
_drop2_=0.7
_epochs_=9500

In [25]:
import wandb

In [26]:
config = {"lr": _lr_, "batch_size": _batch_size_,"architecture": "LSTM", 
          "depth": 2,
          "layer1":_N1_,  "layer2":_N2_, 
          "drop1":_drop1_,"drop2":_drop2_,
          "epochs":_epochs_
          
          
          }

In [27]:



wandb.init(project="pid_3d_lstm", entity="sipoczlaszlo",config=config)

wandb: Currently logged in as: sipoczlaszlo (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.15.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [35]:
import tensorflow as tf


In [36]:
from tensorflow import keras
import wandb 

In [37]:
#from wandb.keras import WandbMetricsLogger
fname="./model_PID_NN_LSTM_"
callbacks = [#callback_LR,
        
        #wandb.tensorflow.WandbHook(steps_per_log=1000),
        tf.keras.callbacks.ModelCheckpoint(filepath=fname+"_{epoch}"+"_{loss:.4f}_{val_loss:.4f}_{val_accuracy:.4f}.hdf5", monitor='val_accuracy',
                        verbose=2, save_best_only=True, mode='max')]

In [38]:
from keras.engine.base_layer import regularizers
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout
from keras.models import Sequential, Model
from keras.optimizers import SGD,Adam,Adamax,Nadam,Ftrl,Adadelta
import tensorflow as tf
#from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
#from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld
from sklearn.model_selection import train_test_split
from keras.callbacks import LearningRateScheduler

clear_session()

kernel_reg_1=tf.keras.regularizers.L2(0.1)

input_size=20


input1=Input(shape=(input_size,3))
l1_out=LSTM(input_size*2,)(input1) 
l2_out=Dropout(_drop1_)(l1_out)


l3_out=Dense(_N2_,activation="swish",kernel_initializer='glorot_uniform',)(l2_out) #kernel_initializer='lecun_normal',
l4_out=Dropout(_drop2_)(l3_out)

pred=Dense(1, activation="sigmoid",)(l4_out)

model = Model(inputs=input1, outputs=pred)
optimizer=Adamax(learning_rate=_lr_,) #

model.compile(loss='binary_crossentropy',
    optimizer=optimizer,
        metrics=["accuracy"])

In [39]:
!rm *.hdf5

In [40]:
X_train,X_test,y_train,y_test=train_test_split(X_all,y_all,train_size=0.9,shuffle=True,random_state=33)

In [41]:
history = model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_test,y_test),
    epochs=1500,
    batch_size=10,
    callbacks=callbacks
   )

A streamkimeneten csak az utolsó 5000 sor látható.
Epoch 251/1500
509/510 [============================>.] - ETA: 0s - loss: 0.1251 - accuracy: 0.9499
Epoch 251: val_accuracy did not improve from 0.95583
510/510 [==============================] - 4s 7ms/step - loss: 0.1251 - accuracy: 0.9499 - val_loss: 0.1332 - val_accuracy: 0.9488
Epoch 252/1500
510/510 [==============================] - ETA: 0s - loss: 0.1237 - accuracy: 0.9484
Epoch 252: val_accuracy did not improve from 0.95583
510/510 [==============================] - 3s 7ms/step - loss: 0.1237 - accuracy: 0.9484 - val_loss: 0.1293 - val_accuracy: 0.9470
Epoch 253/1500
501/510 [============================>.] - ETA: 0s - loss: 0.1313 - accuracy: 0.9473
Epoch 253: val_accuracy did not improve from 0.95583
510/510 [==============================] - 3s 5ms/step - loss: 0.1303 - accuracy: 0.9480 - val_loss: 0.1320 - val_accuracy: 0.9435
Epoch 254/1500
502/510 [============================>.] - ETA: 0s - loss: 0.1267 - accuracy: 0.94